<a href="https://colab.research.google.com/github/LohitSubodh/NSE-market-data/blob/main/OI_NSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<ipython-input-9-b1da491cb404>, line 16)

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=d20d76bdfefe6dd60673d6a1ff2bc7e6f1ca5e438071f6729faa88263537de52
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [8]:
import requests
import plotly.graph_objects as go

# Placeholder variables
symbol = "NIFTY"
selected_expiry_date = "04-Jul-2024"  # Replace with your desired expiry date
lot_size = 50  # Replace with your desired lot size

# The general URL for fetching NIFTY options data
url = f"https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"

# Create a session object
with requests.Session() as session:
    session.headers.update({
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'DNT': '1',
        'Upgrade-Insecure-Requests': '1',
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9,kn;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        'Sec-Fetch-User': '?1',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-Mode': 'navigate'
    })

# First request to get session cookies
session.get("https://www.nseindia.com")

# Now make your API request
response = session.get(url)

# Initialize lists to store the parsed data
strike_prices = []
ce_open_interests = []
pe_open_interests = []

if response.status_code == 200:
    data = response.json()  # Parse the JSON data from the response
    underlying_value = data['records']['data'][0]['PE']['underlyingValue']
    lower_bound = underlying_value - 1000
    upper_bound = underlying_value + 1000
    print(f"Lower Bound: {lower_bound}")
    print(f"Upper Bound: {upper_bound}")
    print(f"underlying value: {underlying_value}")
    # Filter the required data for the specific expiry date
    for record in data['records']['data']:
        if record['expiryDate'] == selected_expiry_date:
            if record['strikePrice'] >= lower_bound and record['strikePrice'] <= upper_bound:
              if record['strikePrice'] %100 == 0:
                strike_prices.append(record['strikePrice'])
            # Check if CE and PE data is available and multiply by lot size
                if 'CE' in record:
                  ce_open_interests.append((record['CE']['openInterest'] * lot_size) / 1e3)  # Convert to millions
                else:
                  ce_open_interests.append(0)
                if 'PE' in record:
                  pe_open_interests.append((record['PE']['openInterest'] * lot_size) / 1e3)  # Convert to millions
                else:
                  pe_open_interests.append(0)
else:
    print("Failed to retrieve data:", response.status_code)

# Creating traces for the Plotly graph
trace1 = go.Bar(
    x=strike_prices,
    y=ce_open_interests,
    name='CE Open Interest'
)
trace2 = go.Bar(
    x=strike_prices,
    y=pe_open_interests,
    name='PE Open Interest'
)

# Creating the layout for the Plotly graph with increased size
layout = go.Layout(
    title=f'NIFTY Call and Put Open Interest for {selected_expiry_date} (Lot Size: {lot_size})',
    xaxis=dict(
        title='Strike Price',
        tickmode='array',
        tickvals=strike_prices[::1]  # Show every 5th strike price for readability
    ),
    yaxis=dict(
        title='Open Interest (in thousands)'
    ),
    barmode='group',
    width=1200,  # Width of the figure in pixels
    height=800   # Height of the figure in pixels
)

# Creating the figure with the data and layout
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Display the plot
fig.show()


Lower Bound: 23286.5
Upper Bound: 25286.5
underlying value: 24286.5
